# **GIT SETUP**

In [1]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)

/content/drive
Mounted at /content/drive


**Change to your custom path**

In [2]:
MY_GOOGLE_DRIVE_PATH = 'My Drive/Capstone_Pritam/'

**Setup directories**

In [3]:

GIT_TOKEN = '4f61cb936944e66abd13e7e809e199ee3a378fae'
GIT_USERNAME = 'mohitgulla' 
GIT_REPOSITORY = 'Edge'

from os.path import join  

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# # In case we haven't created the folder already; we will create a folder in the project path 
!mkdir -p "{PROJECT_PATH}"    

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive/My Drive/Capstone_Pritam/
GIT_PATH:  https://4f61cb936944e66abd13e7e809e199ee3a378fae@github.com/mohitgulla/Edge.git


**Change dirs and clone repo (if needed !)**

In [4]:
%cd "{PROJECT_PATH}"
#!git clone -b Pritam "{GIT_PATH}"
%cd "Edge"

/content/drive/My Drive/Capstone_Pritam
/content/drive/My Drive/Capstone_Pritam/Edge


# **Model = DNN, Data = Telescope data**

In [8]:
# %%writefile train_telescope.py
import time
import torch
import torch.nn as nn
from model.dnn import DenseNeuralNet
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split



def evaluate(model, test_set, batch_size, criterion, ep):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
   

  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  for step, inputs in enumerate(test_iterator):
      global_step +=1
      x = inputs[0]
      y = torch.squeeze(inputs[1],1).long()
      x = x.to(device)
      y = y.to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss

  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  criterion = nn.CrossEntropyLoss()

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  
  train_log = open("log/train.log", "w")
  val_log = open("log/val.log", "w")
  test_log = open("log/test.log", "w")
  

  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    for step, inputs in enumerate(train_iterator):
      model.train()
      model.zero_grad()

      global_step +=1
      # if global_step > 10:
      #   break
      x = inputs[0]
      y = torch.squeeze(inputs[1],1).long()
      x = x.to(device)
      y = y.to(device)

      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()

      
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %f, training accuracy = %f" %(global_step, train_loss, train_accuracy))

    print("Step = %d, validation loss =  %f, validation accuracy = %f" %(global_step, val_loss, val_accuracy))
    
  test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
  test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
  print("End of Training, test loss =  %f, test accuracy = %f" %(test_loss, test_accuracy))

  train_log.close()
  val_log.close()
  test_log.close()

def main():
## main
  input_dim =  10
  output_classes = 2
  learning_rate = 0.001
  batch_size = 16
  epochs = 10
  eval_steps = 100
  ####

  churn_dataset = TelescopeDataset()
  train_set, val_set, test_set = get_get_train_val_test(churn_dataset, val_split=0.40)


  model = DenseNeuralNet(input_dim, output_classes)
  criterion = nn.CrossEntropyLoss()


  train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps)


if __name__ == "__main__":
  main()





Step = 714, validation loss =  0.424105, validation accuracy = 0.810988



Step = 1428, validation loss =  0.401897, validation accuracy = 0.827550



Step = 2142, validation loss =  0.390370, validation accuracy = 0.832545



Step = 2856, validation loss =  0.379529, validation accuracy = 0.842797



Step = 3570, validation loss =  0.372762, validation accuracy = 0.845952


...


Step = 4998, validation loss =  0.367671, validation accuracy = 0.846477



Step = 5712, validation loss =  0.361992, validation accuracy = 0.854101



Step = 6426, validation loss =  0.358923, validation accuracy = 0.852787



Step = 7140, validation loss =  0.355979, validation accuracy = 0.853049




End of Training, test loss =  0.335942, test accuracy = 0.866982
